Download the trained weights for plitterStreet detections

In [ ]:
# !git clone https://github.com/gicait/pLitter
# !pip install /content/pLitter

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1lRKR5SfjhmaWichmBg2TWPufUVDed4zF' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1lRKR5SfjhmaWichmBg2TWPufUVDed4zF" -O /content/yolov5l.pt && rm -rf /tmp/cookies.txt

GoPro videos not working properly with opencv

use ffmpeg to copy the video track to a new file 


In [ ]:
# %cd 'directory_to_videos'

# !for FILE in *.MP4; do ffmpeg -i $FILE -c:v copy "target_dir_to_save_video_tracks"/${FILE//MP4/mp4} -y; done

In [ ]:
import os, cv2, json, time
import torch
# from plitterstreet.predict import draw_boxes_on_image

In [ ]:
colors = [(0,255,255), (0,0,255), (255,0,0), (0,255,0)]
def draw_boxes_on_image(image, boxes, classes, class_ids, scores, use_normalized_coordinates=True, min_score_thresh=.3):
    assert len(boxes) == len(scores) == len(classes)
    for i in range(len(boxes)):
        box = boxes[i]
        category = str(classes[i])
        class_id = int(class_ids[i])
        score = scores[i]
        if score >= min_score_thresh:
          if use_normalized_coordinates:
            h,w,_ = image.shape
            y1 = int(box[0]*h)
            x1 = int(box[1]*w)
            y2 = int(box[2]*h)
            x2 = int(box[3]*w)  
          else:
            x1,y1,x2,y2 = int(box[0]), int(box[1]), int(box[2]), int(box[3])
          image = cv2.rectangle(image, (x1, y1), (x2, y2), colors[class_id], 2)
          cv2.putText(image, category+':'+str(round(score,2)), (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, colors[class_id], 1)
    return image

In [ ]:
weights = '/content/yolov5l.pt'
model = torch.hub.load('ultralytics/yolov5', 'custom', weights, force_reload=True)

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2022-10-9 Python-3.7.14 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
YOLOv5l summary: 367 layers, 46124433 parameters, 0 gradients
Adding AutoShape... 


In [ ]:
# img = cv2.imread('/content/drive/MyDrive/roxana-charles-st-gopro-capture-sept30/GH010017/GH010017_10.jpg')
# preds = model(img, size=1280)
# class_ids = preds.xyxy[0].cpu().numpy()[:, 5]
# classes = [preds.names[cls] for cls in class_ids]
# scores = preds.xyxy[0].cpu().numpy()[:, 4]
# boxes = preds.xyxy[0].cpu().numpy()[:,:4]
# img = draw_boxes_on_image(img, boxes, classes, class_ids, scores, use_normalized_coordinates=False, min_score_thresh=.4)

# from google.colab.patches import cv2_imshow
# cv2_imshow(img)

In [ ]:
source = 'folder_to_videos_to_detect'
target = 'folder_to_save_detected_videos'

# if source == target:
#   print('source should not be same as )

if os.path.isfile(source):
  dataset = [os.path.basename(source)]
  source = os.path.dirname(source)
elif os.path.isdir(source):
  dataset = sorted([im for im in os.listdir(source) if im.lower().endswith(('.mp4', '.avi', '.jpg', '.png'))])
else:
  print('please check source is correct')

print(dataset)

for fn in dataset[-2:-1]:
  print(fn)
  if fn.endswith(('.jpg', '.png')):
    img = cv2.imread(os.path.join(source, fn))
    preds = model(img, size=1280)
    class_ids = preds.xyxy[0].cpu().numpy()[:, 5]
    classes = [preds.names[cls] for cls in class_ids]
    scores = preds.xyxy[0].cpu().numpy()[:, 4]
    boxes = preds.xyxy[0].cpu().numpy()[:,:4]
    img = draw_boxes_on_image(img, boxes, classes, class_ids, scores, use_normalized_coordinates=False, min_score_thresh=.4)
    cv2.imwrite(os.path.join(target, fn), img)
    pred_json = {'file_name': fn, 'category_ids': class_ids, 'boxes': boxes, 'scores': scores}
    json.dump(pred_json, os.path.join(target, fn.replace(('.jpg', '.png'), '.json')))
  else:
    st = time.time()
    vid = cv2.VideoCapture(os.path.join(source, fn))
    width  = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(vid.get(cv2.CAP_PROP_FPS))
    # print(width, height)
    # fn_r = os.path.join(target, fn.split('.')[0]+'.avi')
    fn_r = os.path.join(target, fn)
    print(fn_r)
    # result = cv2.VideoWriter(fn_r, cv2.VideoWriter_fourcc(*'MJPG'), fps, (width, height))
    result = cv2.VideoWriter(fn_r, cv2.VideoWriter_fourcc(*'MP4V'), fps, (width, height))
    # result = cv2.VideoWriter(fn_r, cv2.VideoWriter_fourcc(*'H264'), fps, (width, height))
    pred_json = {'video': fn, 'preds': []}
    c = 1
    while True:
        ret, img = vid.read()
        if img is None:
          json.dump(pred_json, open(target+fn.replace('.MP4', '_result.json'), 'w'))
          result.release()
          vid.release()
          break
        preds = model(img, size=1280)
        class_ids = preds.xyxy[0].cpu().numpy()[:, 5]
        classes = [preds.names[cls] for cls in class_ids]
        scores = preds.xyxy[0].cpu().numpy()[:, 4]
        boxes = preds.xyxy[0].cpu().numpy()[:,:4]
        im = draw_boxes_on_image(img, boxes, classes, class_ids, scores, use_normalized_coordinates=False, min_score_thresh=.4)
        # cv2.imwrite('/content/frames/'+str(c)+'.jpg', im)
        pred_json['preds'].append({'frame': c, 'category_ids': class_ids.tolist(), 'boxes': boxes.tolist(), 'scores': scores.tolist()})
        # print(img.shape)
        result.write(im)
        c+=1
    print(time.time()-st)

convert the results to geojson for plotting on map

In [ ]:
# covert function
# To Do

Testing sorting below

In [ ]:
# !rm -rf Yolov5_StrongSORT_OSNet
# !git clone --recurse-submodules https://github.com/mikel-brostrom/Yolov5_StrongSORT_OSNet.git
# %cd Yolov5_StrongSORT_OSNet
# !pip install -r requirements.txt

In [ ]:
# !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1Kkx2zW89jq_NETu4u42CFZTMVD5Hwm6e' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1Kkx2zW89jq_NETu4u42CFZTMVD5Hwm6e" -O /content/osnet_x0_25_msmt17.pt && rm -rf /tmp/cookies.txt

In [ ]:
# %cd /content

/content


In [ ]:
# # limit the number of cpus used by high performance libraries
# os.environ["OMP_NUM_THREADS"] = "1"
# os.environ["OPENBLAS_NUM_THREADS"] = "1"
# os.environ["MKL_NUM_THREADS"] = "1"
# os.environ["VECLIB_MAXIMUM_THREADS"] = "1"
# os.environ["NUMEXPR_NUM_THREADS"] = "1"

# import sys

# if '/content/Yolov5_StrongSORT_OSNet' not in sys.path:
#     sys.path.append('/content/Yolov5_StrongSORT_OSNet')
# if '/content/Yolov5_StrongSORT_OSNet/strong_sort' not in sys.path:
#     sys.path.append('/content/Yolov5_StrongSORT_OSNet/strong_sort')
# if '/content/Yolov5_StrongSORT_OSNet/strong_sort/deep/reid' not in sys.path:
#     sys.path.append('/content/Yolov5_StrongSORT_OSNet/strong_sort/deep/reid')

# import numpy as np
# from pathlib import Path
# import torch
# import torch.backends.cudnn as cudnn

# from strong_sort.utils.parser import get_config
# from strong_sort.strong_sort import StrongSORT

# config_strongsort = '/content/Yolov5_StrongSORT_OSNet/strong_sort/configs/strong_sort.yaml'
# # strong_sort_weights='/content/osnet_x0_25_msmt17.pt'
# strong_sort_weights=Path('.')/'osnet_x0_25_msmt17.pt'

# # device=torch.device('cpu')
# device=torch.device(0)
# half=False

# # initialize StrongSORT
# cfg = get_config()
# cfg.merge_from_file(config_strongsort)

# source = '/content/drive/MyDrive/roxana-charles-st-gopro-capture-sept30'
# target = '/content/drive/MyDrive/pLitter/Street/Boston/output'

# # if source == target:
# #   print('source should not be same as )

# if os.path.isfile(source):
#   dataset = [os.path.basename(source)]
#   source = os.path.dirname(source)
# elif os.path.isdir(source):
#   dataset = sorted([im for im in os.listdir(source) if im.lower().endswith(('.mp4', '.avi', '.jpg', '.png'))])
# else:
#   print('please check source is correct')

# print(dataset)

# with torch.no_grad():

#     for fn in dataset[-2:-1]:
#       print(fn)
#       if fn.lower().endswith(('.mp4', '.avi')):
#         strongsort = StrongSORT(
#             strong_sort_weights,
#             device,
#             half,
#             max_dist=cfg.STRONGSORT.MAX_DIST,
#             max_iou_distance=cfg.STRONGSORT.MAX_IOU_DISTANCE,
#             max_age=cfg.STRONGSORT.MAX_AGE,
#             n_init=cfg.STRONGSORT.N_INIT,
#             nn_budget=cfg.STRONGSORT.NN_BUDGET,
#             mc_lambda=cfg.STRONGSORT.MC_LAMBDA,
#             ema_alpha=cfg.STRONGSORT.EMA_ALPHA,
#         )
#         strongsort.model.warmup()
#         outputs = []
#         prev_frame = None
        
#         st = time.time()
#         vid = cv2.VideoCapture(os.path.join(source, fn))
#         width  = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
#         height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
#         fps = int(vid.get(cv2.CAP_PROP_FPS))
#         fn_r = os.path.join(target, fn)
#         print(fn_r)
#         result = cv2.VideoWriter(fn_r, cv2.VideoWriter_fourcc(*'MP4V'), fps, (width, height))
#         pred_json = {'video': fn, 'preds': []}
#         c = 1
#         while True:
#             ret, img = vid.read()
#             if img is None:
#               json.dump(pred_json, open(target+fn.replace('.MP4', '_result.json'), 'w'))
#               result.release()
#               vid.release()
#               break
#             preds = model(img, size=1024)
#             # class_ids = preds.xyxy[0].cpu().numpy()[:, 5]
#             #classes = [preds.names[cls] for cls in class_ids]
#             # scores = preds.xyxy[0].cpu().numpy()[:, 4]
#             #boxes = preds.xyxy[0].cpu().numpy()[:,:4]
#             boxes = preds.xywh[0][0:,:4]
#             scores = preds.xywh[0][:, 4]
#             class_ids = preds.xywh[0][:, 5]
#             # print(xywhs, confs, clss)
#             curr_frame = img
#             # print(type(img), img.shape)
#             if cfg.STRONGSORT.ECC:  # camera motion compensation
#               strongsort.tracker.camera_update(prev_frame, curr_frame)
#             print(boxes.cpu(), scores.cpu(), class_ids.cpu(), img.shape)
#             outputs = strongsort.update(boxes.cpu(), scores.cpu(), class_ids.cpu(), img)
#             print(len(outputs))
#             if len(outputs)>0:
#               im = draw_boxes_on_image(img, boxes, classes, class_ids, scores, use_normalized_coordinates=False, min_score_thresh=.4)
#             else:
#               im = img
#               strongsort.increment_ages()
#             pred_json['preds'].append({'frame': c, 'category_ids': class_ids.tolist(), 'boxes': boxes.tolist(), 'scores': scores.tolist()})
#             result.write(im)
#             c+=1
#             prev_frame = curr_frame
#         print(time.time()-st)

In [ ]:
# %cd Yolov5_StrongSORT_OSNet
# !python track.py --yolo-weights /content/yolov5l.pt --strong-sort-weights osnet_x0_25_msmt17.pt --source /content/drive/MyDrive/roxana-charles-st-gopro-capture-sept30/GH010021.mp4 --save-vid

In [ ]:
# # limit the number of cpus used by high performance libraries
# os.environ["OMP_NUM_THREADS"] = "1"
# os.environ["OPENBLAS_NUM_THREADS"] = "1"
# os.environ["MKL_NUM_THREADS"] = "1"
# os.environ["VECLIB_MAXIMUM_THREADS"] = "1"
# os.environ["NUMEXPR_NUM_THREADS"] = "1"

# import sys

# if '/content/Yolov5_StrongSORT_OSNet' not in sys.path:
#     sys.path.append('/content/Yolov5_StrongSORT_OSNet')
# if '/content/Yolov5_StrongSORT_OSNet/yolov5' not in sys.path:
#   sys.path.append('/content/Yolov5_StrongSORT_OSNet/yolov5')
# if '/content/Yolov5_StrongSORT_OSNet/strong_sort' not in sys.path:
#     sys.path.append('/content/Yolov5_StrongSORT_OSNet/strong_sort')
# if '/content/Yolov5_StrongSORT_OSNet/strong_sort/deep/reid' not in sys.path:
#     sys.path.append('/content/Yolov5_StrongSORT_OSNet/strong_sort/deep/reid')

# import numpy as np
# from pathlib import Path
# import torch
# import torch.backends.cudnn as cudnn

# from strong_sort.utils.parser import get_config
# from strong_sort.strong_sort import StrongSORT
# import logging
# from yolov5.models.common import DetectMultiBackend
# from yolov5.utils.dataloaders import VID_FORMATS, LoadImages, LoadStreams
# from yolov5.utils.general import (LOGGER, check_img_size, non_max_suppression, scale_coords, check_requirements, cv2,
#                                   check_imshow, xyxy2xywh, increment_path, strip_optimizer, colorstr, print_args, check_file)
# from yolov5.utils.torch_utils import select_device, time_sync
# from yolov5.utils.plots import Annotator, colors, save_one_box
# from strong_sort.utils.parser import get_config
# from strong_sort.strong_sort import StrongSORT


# # @torch.no_grad()
# with torch.no_grad():

#     yolo_weights='/content/yolov5l.pt'
#     strong_sort_weights=Path('/content/Yolov5_StrongSORT_OSNet/osnet_x0_25_msmt17.pt')
#     config_strongsort='/content/Yolov5_StrongSORT_OSNet/strong_sort/configs/strong_sort.yaml'
#     imgsz=(1024, 1024)
#     conf_thres=0.25
#     iou_thres=0.45
#     max_det=1000
#     device=0
#     classes=None
#     agnostic_nms=False
#     augment=False
#     visualize=False
#     update=False
#     exist_ok=False
#     half=False
#     dnn=False
#     eval=False

#     source = '/content/drive/MyDrive/roxana-charles-st-gopro-capture-sept30'


#     # print(torch.device(0))

#     device = torch.device(0)
#     print(yolo_weights)
#     model = DetectMultiBackend(yolo_weights, device=device, dnn=dnn, data=None, fp16=half)
#     stride, names, pt = model.stride, model.names, model.pt
#     imgsz = check_img_size(imgsz, s=stride)  # check image size

#     dataset = LoadImages(source, img_size=imgsz, stride=stride, auto=pt)
#     nr_sources = 1
#     vid_path, vid_writer, txt_path = [None] * nr_sources, [None] * nr_sources, [None] * nr_sources

#     # initialize StrongSORT
#     cfg = get_config()
#     cfg.merge_from_file(config_strongsort)

#     # Create as many strong sort instances as there are video sources
#     strongsort_list = []
#     for i in range(nr_sources):
#         strongsort_list.append(
#             StrongSORT(
#                 strong_sort_weights,
#                 device,
#                 half,
#                 max_dist=cfg.STRONGSORT.MAX_DIST,
#                 max_iou_distance=cfg.STRONGSORT.MAX_IOU_DISTANCE,
#                 max_age=cfg.STRONGSORT.MAX_AGE,
#                 n_init=cfg.STRONGSORT.N_INIT,
#                 nn_budget=cfg.STRONGSORT.NN_BUDGET,
#                 mc_lambda=cfg.STRONGSORT.MC_LAMBDA,
#                 ema_alpha=cfg.STRONGSORT.EMA_ALPHA,

#             )
#         )
#         strongsort_list[i].model.warmup()
#     outputs = [None] * nr_sources

#     # Run tracking
#     model.warmup(imgsz=(1 if pt else nr_sources, 3, *imgsz))  # warmup
#     dt, seen = [0.0, 0.0, 0.0, 0.0], 0
#     curr_frames, prev_frames = [None] * nr_sources, [None] * nr_sources
#     for frame_idx, (path, im, im0s, vid_cap, s) in enumerate(dataset):
#         t1 = time_sync()
#         im = torch.from_numpy(im).to(device)
#         im = im.half() if half else im.float()  # uint8 to fp16/32
#         im /= 255.0  # 0 - 255 to 0.0 - 1.0
#         if len(im.shape) == 3:
#             im = im[None]  # expand for batch dim
#         t2 = time_sync()
#         dt[0] += t2 - t1

#         # Inference
#         visualize = increment_path(save_dir / Path(path[0]).stem, mkdir=True) if visualize else False
#         pred = model(im, augment=augment, visualize=visualize)
#         t3 = time_sync()
#         dt[1] += t3 - t2

#         # Apply NMS
#         pred = non_max_suppression(pred, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det)
#         dt[2] += time_sync() - t3

#         # Process detections
#         for i, det in enumerate(pred):  # detections per image
#             seen += 1
#             p, im0, _ = path, im0s.copy(), getattr(dataset, 'frame', 0)
#             p = Path(p)  # to Path
#             # video file
#             if source.endswith(VID_FORMATS):
#                 txt_file_name = p.stem
#                 save_path = str(save_dir / p.name)  # im.jpg, vid.mp4, ...
#             # folder with imgs
#             else:
#                 txt_file_name = p.parent.name  # get folder name containing current img
#             curr_frames[i] = im0

#             s += '%gx%g ' % im.shape[2:]  # print string
#             # imc = im0.copy() if save_crop else im0  # for save_crop

#             # annotator = Annotator(im0, line_width=line_thickness, pil=not ascii)
#             if cfg.STRONGSORT.ECC:  # camera motion compensation
#                 strongsort_list[i].tracker.camera_update(prev_frames[i], curr_frames[i])

#             if det is not None and len(det):
#                 # Rescale boxes from img_size to im0 size
#                 det[:, :4] = scale_coords(im.shape[2:], det[:, :4], im0.shape).round()

#                 # Print results
#                 for c in det[:, -1].unique():
#                     n = (det[:, -1] == c).sum()  # detections per class
#                     s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # add to string

#                 xywhs = xyxy2xywh(det[:, 0:4])
#                 confs = det[:, 4]
#                 clss = det[:, 5]

#                 print(xywhs.cpu(), confs.cpu(), clss.cpu())

#                 # pass detections to strongsort
#                 t4 = time_sync()
#                 outputs[i] = strongsort_list[i].update(xywhs.cpu(), confs.cpu(), clss.cpu(), im0)
#                 t5 = time_sync()
#                 dt[3] += t5 - t4

#                 # draw boxes for visualization
#                 if len(outputs[i]) > 0:
#                     for j, (output, conf) in enumerate(zip(outputs[i], confs)):

#                         bboxes = output[0:4]
#                         id = output[4]
#                         cls = output[5]


#             else:
#                 strongsort_list[i].increment_ages()
#             prev_frames[i] = curr_frames[i]

#     # Print results
#     t = tuple(x / seen * 1E3 for x in dt)  # speeds per image
#     if update:
#         strip_optimizer(yolo_weights)  # update model (to fix SourceChangeWarning)